# Epoch test
This script 

In [ ]:
# Funky shit
import os
import csv
os.environ['LD_LIBRARY_PATH'] = '/usr/lib/x86_64-linux-gnu/libcudart.so.11.0'
from PIL import Image,ImageOps
from tqdm.notebook import tqdm

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf

import numpy as np
import matplotlib.pyplot as plt

## Import MNIST Fashion data

In [ ]:
# fashion_mnist = tf.keras.datasets.fashion_mnist

# (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# # Setup labels as vector outputs rather than digit value
# train_labelsVektor = np.zeros((10, len(train_labels)))
# for i in range(len(train_labels)):
#     for j in range(10):
#         if train_labels[i] == j:
#             train_labelsVektor[j, i] += 1
# train_labelsVektor = np.transpose(train_labelsVektor)

# test_labelsVektor = np.zeros((10, len(test_labels)))
# for i in range(len(test_labels)):
#     for j in range(10):
#         if test_labels[i] == j:
#             test_labelsVektor[j, i] += 1
# test_labelsVektor = np.transpose(test_labelsVektor)

# class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
#                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# # Normalize
# train_images = train_images / 255.0
# test_images = test_images / 255.0

In [ ]:
def load_reduced_data(keepClasses: list, qubitCount):
    """
    Takes a list of classes to keep and returns reduced mnist dataset.
    Loops over full data and appends to list if the class is in keepClasses.
    A list is used for performance reasons, since Numpy Append copies the entire array
    every time. The lists are cast to numpy array before returning.
    
    :param list keepClasses: List of ints.
    :return: returns two tuples. One with train data and one with test data.
    """
    transDict = {}
    for i in range(len(keepClasses)):
        transDict[keepClasses[i]] = i
    print(transDict)
    
    
    #fashion_mnist = fashion_mnist
    (train_images_full, train_labels_full), (test_images_full, test_labels_full) = tf.keras.datasets.fashion_mnist.load_data()
    
    train_images = []
    train_labels = []
    test_images = []
    test_labels = []
    print("Loading train data")
    for i in tqdm(list(range(len(train_labels_full)))):
        if train_labels_full[i] in keepClasses:
            train_labels.append(transDict[train_labels_full[i]])
            train_images.append(train_images_full[i])
    train_images = np.array(train_images)
    train_labels = np.array(train_labels)
    print("Loading test data")
    for i in tqdm(list(range(len(test_labels_full)))):
        if test_labels_full[i] in keepClasses:
            test_labels.append(transDict[test_labels_full[i]])
            test_images.append(test_images_full[i])
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)
    
    
    # Setup labels as vector outputs rather than digit value
    train_labelsVektor = np.zeros((qubitCount, len(train_labels)))
    for i in range(len(train_labels)):
        for j in range(len(keepClasses)):
            if train_labels[i] == j:
                train_labelsVektor[j, i] += 1
    train_labelsVektor = np.transpose(train_labelsVektor)

    test_labelsVektor = np.zeros((qubitCount, len(test_labels)))
    for i in range(len(test_labels)):
        for j in range(len(keepClasses)):
            if test_labels[i] == j:
                test_labelsVektor[j, i] += 1
    test_labelsVektor = np.transpose(test_labelsVektor)
    
    train_images = train_images / 255.0
    test_images = test_images / 255.0
    
    return (train_images, train_labelsVektor), (test_images, test_labelsVektor)

In [ ]:
def nDimPCA(dimOut, data):
    """
    Applies the PCA with to training data 
    
    :param int dimOut: Dimension of the output data
    :param np.array data: Matrices containing the training data
    :return: returns the reduced training data now in reduced dimension
    """
    #Normalize data
    #Flatten input data
    data = data.reshape([np.size(data,0),np.size(data,1)*np.size(data,2)])
    #Mean
    mu = np.mean(data, axis = 0)
    mu = mu.reshape([np.size(mu,0),1])
    #covariance
    data = np.transpose(data)
    sigma = np.cov(data-mu)
    #SVD
    U, S, V = np.linalg.svd(sigma)
    #V reduced
    VReduced = V[0:dimOut,0:]
    #output reduced data
    out = np.dot(VReduced,(data-mu))
    return np.transpose(out),mu,VReduced

In [ ]:
def PCAOnTest(VReduced, mu, data):
    """
    Applies the PCA to test data
    
    :param np.array VReduced: Eigenvectors for the PCA
    :param np.array mu: Vector containing the mean values 
    :param np.array data: Matrices containing the test data
    :return: returns the reduced test data now in the same dimension as the training data
    """
    data = data.reshape([np.size(data,0),np.size(data,1)*np.size(data,2)])
    data = np.transpose(data)
    out = np.dot(VReduced,(data-mu))
    
    
    return np.transpose(out)

In [ ]:
def generateModel(nodes, ActivationFunctions, inputShape, outputShape):
    model = tf.keras.Sequential()
    #model.add(tf.keras.layers.InputLayer(batch_size=32))
    for i in range(len(nodes)):
        model.add(tf.keras.layers.Dense(nodes[i], activation=activationFunctions))
    model.add(tf.keras.layers.Dense(outputShape, activation='softmax'))
    return model

In [ ]:
def compileModel(model):
    model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['accuracy'])
    return model

In [ ]:
def trainModel(model, epochs):
    trainAccuracy = np.zeros(epochs)
    testAccuracy = np.zeros(epochs)
    for i in range(epochs):
        print('Epoch iteration: ' + str(i+1))
        with tf.device('/GPU:0'):
            history = model.fit(train_images_red, train_labelsVektor, epochs=1)
        trainAccuracy[i] = history.history["accuracy"][0]
        test_loss, test_acc = model.evaluate(test_images_red,  test_labelsVektor)
        testAccuracy[i] = test_acc
    return trainAccuracy, testAccuracy

In [ ]:
def writeToCSV(nodes, dimSet, activationFunction, trainAccuracy, testAccuracy, classes):
    with open('./Verysmallnetwork/' + 'PCA' + str(dimSet) + 'NODES' + str(nodes) + 'classes' + str(len(classes)) +'.csv', 'w', encoding='UTF8') as f: #/PCA_DNN_test
        writer = csv.writer(f)
        writer.writerow(['Layers: ' + str(len(nodes))])
        writer.writerow(['Nodes: ' + str(nodes)])
        writer.writerow(['PCA dim: ' + str(dimSet)])
        writer.writerow(['Activation function: ' + activationFunction])
        writer.writerow(['Training accuracy', 'Test accuracy'])
        writer.writerow(['Classes: ' + str(classes)])
        for i in range(len(trainAccuracy)):
            writer.writerow([trainAccuracy[i], testAccuracy[i]])

In [ ]:
# load data
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
(train_images, train_labelsVektor), (test_images, test_labelsVektor) = load_reduced_data(classes, len(classes))

# Neural network setup
nodes = [[1],[2],[3],[4],[5],[6],[7],[8],[9],[10],[11],[12]]
activationFunctions = 'relu'
epochs = 1
##

# PCA thetas list
# dimSet = [5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 125, 150, 175, 200, 300, 400]
dimSet = [20]

for i in range(len(dimSet)):
    for j in range(len(nodes)):
        # Reduce data with PCA
        train_images_red, mean, VRed = nDimPCA(dimSet[i], train_images)
        test_images_red = PCAOnTest(VRed, mean, test_images)
        #
        print("PCA dim: " + str(dimSet[i]))
        model = generateModel(nodes[j], activationFunctions, dimSet[i], len(classes))
        model = compileModel(model)
        trainAccuracy, testAccuracy = trainModel(model, epochs)
        #writeToCSV(nodes[j], dimSet[i], activationFunctions, trainAccuracy, testAccuracy, classes)
        print(model.summary())
#model.summary()

In [ ]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))
!jupyter --version